In [ ]:
%matplotlib inline
import math
import numpy as np
import scipy
import scipy.fftpack as fft
import scipy.signal as signal
import matplotlib.pyplot as pp

In [ ]:
X = np.linspace(-math.pi, math.pi, 4096)

n_dot_l = np.maximum(0, np.cos(X))
pp.plot(X, n_dot_l, linewidth=2)

approx = (1/math.pi) + 0.5*np.cos(X) + (2/3/math.pi)*np.cos(2*X)
pp.plot(X, approx)

P = fft.fft(n_dot_l)
# P[3:] = 0
# P[1:] *= 2
ifft = np.real(fft.ifft(P))
pp.plot(X, ifft)

In [ ]:
def convolve(solid_angle):
    n = len(n_dot_l)*solid_angle/2/math.pi
    win = signal.windows.boxcar(max(int(n), 1))
    return signal.convolve(n_dot_l, win/len(win), mode='same')

pp.plot(X, convolve(0.5*math.pi))
pp.plot(X, n_dot_l)

In [ ]:
def convolve_approx(solid_angle):
    y = fft.fft(convolve(solid_angle))
    y[3:] = 0
    y[1:] *= 2
    return np.real(fft.ifft(y))

pp.plot(X, n_dot_l)
pp.plot(X, convolve_approx(0.5*math.pi))

In [ ]:
win = signal.windows.boxcar(16)
win /= len(win)
foo = fft.fft(win, 4096)
foo = fft.fftshift(foo)
pp.plot(np.abs(foo))
pp.plot(np.abs(scipy.special.sinc(X*8/math.pi)))
# Close enough to sinc()?

In [ ]:
n_dot_l = np.maximum(0, np.cos(X))
# pp.plot(X, n_dot_l, color='lightgrey')

approx = (1/math.pi) + 0.5*np.cos(X) + (2/3/math.pi)*np.cos(2*X)
pp.plot(X, approx, color='grey')
pp.plot(X, convolve_approx(math.pi/2**0), color='grey')
pp.plot(X, convolve_approx(math.pi/2**1), color='grey')
pp.plot(X, convolve_approx(math.pi/2**2), color='grey')
pp.plot(X, convolve_approx(math.pi/2**3), color='grey')
pp.plot(X, convolve_approx(math.pi/2**4), color='grey')

bar, sinc = 2*2**1, scipy.special.sinc
a, b, c = sinc(0/bar), sinc(1/bar), sinc(2/bar)
approx = a*(1/math.pi) + b*0.5*np.cos(X) + c*(2/3/math.pi)*np.cos(2*X)
pp.plot(X, approx, color='green')

In [ ]:
x = np.linspace(0, math.pi, 100)
pp.plot(x, np.cos(x))
pp.plot(x, scipy.special.sinc(x/math.pi))

pp.figure()
x = np.linspace(0, math.pi, 100)
pp.plot(np.cos(x), scipy.special.sinc(x/math.pi))
x = np.linspace(-1, 1, 100)
pp.plot(x, (0.5 + 0.5*x)**0.63)
pp.plot(x, np.sqrt(0.5 + 0.5*x))

In [ ]:
def solid_angle_half(d_ratio):
    d = d_ratio
    return np.sqrt(d*d - 1)/d

def solid_angle_full(d_ratio):
    d = d_ratio
    return 2*(d*d - 1)/(d*d) - 1

x = np.linspace(1e-1, 1, 200)
half = np.arccos(solid_angle_half(1/x))
full = np.arccos(solid_angle_full(1/x))
pp.plot(x, half)
pp.plot(x, full)

pp.figure()
pp.axhline(1, color='grey')
pp.plot(x, scipy.special.sinc(half/math.pi))
pp.plot(x, scipy.special.sinc(full/math.pi))

sinc_2 = scipy.special.sinc(0.5)
pp.plot(x, sinc_2 + (1 - sinc_2)*(1 - x**2)**0.5)
pp.plot(x, (1 - x**3))
# pp.plot(1/x, (1 - x**1.9)**0.62)

In [ ]:
def plot_approx(distance):
    radius = 1
    solid_angle = math.acos(solid_angle_full(distance))
    
    pp.figure()
    pp.title(f"Convolution Arc: {round(solid_angle/math.pi, 2)}*π")

    # Reference curve
    pp.plot(X, convolve(solid_angle), linewidth=3)
    # pp.plot(X, convolve_approx(solid_angle), linewidth=3)

    def sinc_pi(x): return scipy.special.sinc(x/math.pi)
    # Approximate main lobe of sinc(x/pi) using cos(x)
    def sinc_pi(x): return math.sqrt(0.5 + 0.5*math.cos(x))
    # A better approximation
    # def sinc_pi(x): return (0.5 + 0.5*math.cos(x))**0.64

    # Direct implementation using sinc_pi()
    # b, c = sinc_pi(0.5*solid_angle), sinc_pi(1*solid_angle)
    # approx = (1/math.pi) + b*0.5*np.cos(X) + c*(2/3/math.pi)*np.cos(2*X)
    # pp.plot(X, approx)

    # Area light convolution in fourier space.
    # Approximation convolution of C1 and C2 with sinc(solid_angle/(3 - c_i))
    dist_ratio = radius/distance
    dist_factor = 1 - dist_ratio*dist_ratio*dist_ratio
    C0 = 0.3183
    C1 = 0.3183 + 0.1817*dist_factor
    C2 = 0.2122*dist_factor
    approx = C0 + C1*np.cos(X) + C2*np.cos(2*X)
    pp.plot(X, approx)

plot_approx(1.0)
plot_approx(1.15)
# plot_approx(math.sqrt(2))
plot_approx(2)
plot_approx(1e3)
